In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
%fs
ls /mnt/olymproj/refined-data

path,name,size,modificationTime
dbfs:/mnt/olymproj/refined-data/atheletes/,atheletes/,0,0
dbfs:/mnt/olymproj/refined-data/coaches/,coaches/,0,0
dbfs:/mnt/olymproj/refined-data/entriesgender/,entriesgender/,0,0
dbfs:/mnt/olymproj/refined-data/medals/,medals/,0,0
dbfs:/mnt/olymproj/refined-data/teams/,teams/,0,0


In [0]:
df1=spark.read.format("delta").load("/mnt/olymproj/refined-data/atheletes/",header="true")
df2=spark.read.format("delta").load("/mnt/olymproj/refined-data/coaches/",header="true")
df3=spark.read.format("delta").load("/mnt/olymproj/refined-data/entriesgender/",header="true")
df4=spark.read.format("delta").load("/mnt/olymproj/refined-data/medals/",header="true")
df5=spark.read.format("delta").load("/mnt/olymproj/refined-data/teams/",header="true")

In [0]:
adf1=df1.groupBy("Country").agg(
count("PersonName").alias("Players_count")).orderBy(expr("Players_Count").desc()).limit(10)
display(adf1)

Country,Players_count
United States of America,615
Japan,586
Australia,470
People's Republic of China,401
Germany,400
France,377
Canada,368
Great Britain,366
Italy,356
Spain,324


In [0]:
df_join=adf1.join(df4, 
               adf1.Country == df4.Team_Country, 
               "inner").orderBy("Rank_Overall")


In [0]:
df6=df_join.select(
    col("Country"),
    col("Gold_won"),
    col("Silver_won"),
    col("Bronze_won"),
    col("Total_won"),
    col("Rank_Overall"),
    col("Players_count")
)
display(df6)

Country,Gold_won,Silver_won,Bronze_won,Total_won,Rank_Overall,Players_count
United States of America,39,41,33,113,1,615
People's Republic of China,38,32,18,88,2,401
Great Britain,22,21,22,65,4,366
Japan,27,14,17,58,5,586
Australia,17,7,22,46,6,470
Italy,10,10,20,40,7,356
Germany,10,11,16,37,8,400
France,10,12,11,33,10,377
Canada,7,6,11,24,11,368
Spain,3,8,6,17,17,324


In [0]:
df6.write.mode("overwrite").option("header",'True').option("path","/mnt/olymproj/reporting-data/Final").save()